In [1]:
import numpy as np
import pandas as pd
import scipy.signal as signal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler




C:\Users\Admin\AppData\Local\Temp\ipykernel_10356\210559016.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'scipy'

In [5]:
import os
from PIL import Image





In [27]:


def read_jpg_files(folder_path):
    # List all files in the directory
    files = os.listdir(folder_path)
    
    # Filter out jpg files
    jpg_files = [file for file in files if file.endswith('.jpg')]
    
    images = []
    for jpg_file in jpg_files:
        # Construct full file path
        file_path = os.path.join(folder_path, jpg_file)
        
        # Open and read the image
        image = Image.open(file_path)
        images.append(image)
    
    return images

# Example usage
folder_path ="C:\\Users\\Admin\\Desktop\\Pogramming\\Arousal\\High"
images = read_jpg_files(folder_path)

# Display the first image
if images:
    images[0].show()
else:
    print("No images found.")



In [28]:
def preprocess_ecg(ecg_signal, fs=128):
    # Bandpass filter the ECG signal
    nyquist = 0.5 * fs
    low = 0.5 / nyquist
    high = 15 / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_ecg = filtfilt(b, a, ecg_signal)
    
    # Pan-Tompkins QRS detection algorithm (simplified version)
    diff = np.diff(filtered_ecg)
    squared = diff ** 2
    integrated = np.convolve(squared, np.ones(15) / 15)
    peaks, _ = signal.find_peaks(integrated, distance=fs//2)  # Assuming a distance between peaks
    
    # Calculate RR intervals
    rr_intervals = np.diff(peaks) / fs  # Convert to seconds
    
    return rr_intervals

def preprocess_gsr(gsr_signal, fs=128):
    # Bandpass filter the GSR signal
    nyquist = 0.5 * fs
    low = 0.05 / nyquist
    high = 19 / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_gsr = filtfilt(b, a, gsr_signal)
    
    # Resample the signal
    resampled_gsr = signal.resample(filtered_gsr, int(len(filtered_gsr) * 10 / fs))  # Resample to 10 Hz
    
    # SCR peak detection (simplified version)
    peaks, _ = signal.find_peaks(resampled_gsr, distance=20)  # Assuming a minimum distance between peaks
    
    return resampled_gsr, peaks


In [33]:
def extract_features(rr_intervals, gsr_peaks, resampled_gsr):
    # ECG features
    ecg_features = {
        'mean_rr': np.mean(rr_intervals),
        'std_rr': np.std(rr_intervals),
        'rmssd': np.sqrt(np.mean(np.square(np.diff(rr_intervals))))
    }
    
    # GSR features
    gsr_amplitudes = resampled_gsr[gsr_peaks]
    gsr_features = {
        'mean_amplitude': np.mean(gsr_amplitudes),
        'std_amplitude': np.std(gsr_amplitudes),
        'num_peaks': len(gsr_peaks)
    }
    
    # Combine features
    features = {**ecg_features, **gsr_features}
    return features

data = pd.DataFrame({
    'ecg_signal': [np.random.rand(1000), np.random.rand(1000)],
    'gsr_signal': [np.random.rand(1000), np.random.rand(1000)],
    'emotion': ['happy', 'sad']
})
# Preprocess dataset and extract features
features_list = []
labels = data['emotion']  # Assuming 'emotion' is the target column

for index, row in data.iterrows():
    ecg_signal = np.array(row['ecg_signal'])  # Assuming ECG signal is stored as an array
    gsr_signal = np.array(row['gsr_signal'])  # Assuming GSR signal is stored as an array
    
    rr_intervals = preprocess_ecg(ecg_signal)
    resampled_gsr, gsr_peaks = preprocess_gsr(gsr_signal)
    
    features = extract_features(rr_intervals, gsr_peaks, resampled_gsr)
    features_list.append(features)

# Create a DataFrame from the features
features_df = pd.DataFrame(features_list)


In [45]:
print(features_df)

     mean_rr    std_rr     rmssd  mean_amplitude  std_amplitude  num_peaks  \
0   0.374540  0.031429  0.642032        0.051682       0.103124          5   
1   0.950714  0.636410  0.084140        0.531355       0.902553          0   
2   0.731994  0.314356  0.161629        0.540635       0.505252          8   
3   0.598658  0.508571  0.898554        0.637430       0.826457          0   
4   0.156019  0.907566  0.606429        0.726091       0.320050          4   
..       ...       ...       ...             ...            ...        ...   
95  0.493796  0.349210  0.522243        0.930757       0.353352          2   
96  0.522733  0.725956  0.769994        0.858413       0.583656          0   
97  0.427541  0.897110  0.215821        0.428994       0.077735          4   
98  0.025419  0.887086  0.622890        0.750871       0.974395          3   
99  0.107891  0.779876  0.085347        0.754543       0.986211          9   

    emotion  
0         1  
1         0  
2         1  
3      

In [46]:
features_df.describe()

,mean_rr,std_rr,rmssd,mean_amplitude,std_amplitude,num_peaks,emotion
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.470181,0.497832,0.517601,0.491149,0.516046,3.750000,0.500000
std,0.297489,0.293111,0.293426,0.293452,0.318601,2.900279,0.502519
min,0.005522,0.006952,0.005062,0.014393,0.010838,0.000000,0.000000
25%,0.193201,0.242005,0.276880,0.249615,0.263935,1.000000,0.000000
50%,0.464142,0.505625,0.562555,0.509718,0.525399,4.000000,0.500000
75%,0.730203,0.766184,0.752367,0.735778,0.796884,6.000000,1.000000
max,0.986887,0.985650,0.990054,0.990505,0.992965,9.000000,1.000000


In [43]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming features_df is the DataFrame with the features
# Mockup data for demonstration
np.random.seed(42)
n_samples = 100
features_df = pd.DataFrame({
    'mean_rr': np.random.rand(n_samples),
    'std_rr': np.random.rand(n_samples),
    'rmssd': np.random.rand(n_samples),
    'mean_amplitude': np.random.rand(n_samples),
    'std_amplitude': np.random.rand(n_samples),
    'num_peaks': np.random.randint(0, 10, n_samples),
    'emotion': np.random.randint(0, 2, n_samples)  # Binary classification
})

# Splitting data into features and labels
X = features_df.drop('emotion', axis=1).values
y = features_df['emotion'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape input data to fit Conv1D layer (samples, time steps, features)
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

def build_dcnn(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
model = build_dcnn(input_shape)

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/10
3/3 [==============================] - 1s 151ms/step - loss: 0.6970 - accuracy: 0.5125 - val_loss: 0.6984 - val_accuracy: 0.3500
Epoch 2/10
3/3 [==============================] - 0s 24ms/step - loss: 0.6841 - accuracy: 0.6250 - val_loss: 0.6995 - val_accuracy: 0.4500
Epoch 3/10
3/3 [==============================] - 0s 21ms/step - loss: 0.6731 - accuracy: 0.5750 - val_loss: 0.6997 - val_accuracy: 0.4500
Epoch 4/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6661 - accuracy: 0.6500 - val_loss: 0.7020 - val_accuracy: 0.4000
Epoch 5/10
3/3 [==============================] - 0s 19ms/step - loss: 0.6595 - accuracy: 0.7000 - val_loss: 0.7050 - val_accuracy: 0.4500
Epoch 6/10
3/3 [==============================] - 0s 21ms/step - loss: 0.6551 - accuracy: 0.6625 - val_loss: 0.7087 - val_accuracy: 0.4500
Epoch 7/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6491 - accuracy: 0.6875 - val_loss: 0.7117 - val_accuracy: 0.4500
Epoch 8/10
3/3 [==========